In [131]:
# Importing all the libraries at once
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display_html
from itertools import chain,cycle

In [132]:
# This function is going to be used to display multiple frames side by side. This will especially help in analyzing pivot tables
# that are created later for data analysis
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [133]:
# this is set so that more rows and more columns can be seen for taking glimpse of data
pd.options.display.max_rows = 50
pd.options.display.max_columns = 150

In [134]:
loans_original = pd.read_csv("loan.csv", low_memory=False)

In [135]:
loan_without_null_coulumns = loans_original.dropna(axis=1, how='all')

In [136]:
for i in loan_without_null_coulumns.columns:
    if(loan_without_null_coulumns[i].nunique() == 1):
        loan_without_null_coulumns = loan_without_null_coulumns.drop(i, axis=1)

In [137]:
loan_without_null_coulumns = loan_without_null_coulumns.drop(['pub_rec_bankruptcies'], axis=1)

In [138]:
loan_without_null_coulumns = loan_without_null_coulumns.drop(['loan_amnt', 'funded_amnt_inv'], axis=1)

In [139]:
loan_without_null_and_behavioral_vals= loan_without_null_coulumns.drop(['delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 
                                                                    'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 
                                                                    'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
                                                                    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 
                                                                    'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 
                                                                    'last_pymnt_amnt', 'last_credit_pull_d', 'next_pymnt_d',
                                                                    'mths_since_last_record'], axis=1)

In [140]:
loan_with_required_cols = loan_without_null_and_behavioral_vals.drop(['id', 'member_id', 'emp_title', 
                                                                        'url', 'desc', 'title']
                                                                     , axis=1)

In [141]:
loan_with_required_cols = loan_with_required_cols[loan_with_required_cols.loan_status != "Current"]

In [142]:
loan_with_required_cols = loan_with_required_cols[~((loan_with_required_cols.home_ownership == "NONE") | 
                                                    (loan_with_required_cols.home_ownership == "OTHER"))]

In [143]:
inc_150k_less = loan_with_required_cols[loan_with_required_cols.annual_inc <= 150000]

In [144]:
###################### Replacing Null in these columns with 0 ######################
values = {'mths_since_last_delinq': 0.0}
inc_150k_less = inc_150k_less.fillna(value=values)

In [145]:
# converting int_rate from string object to float datatype
inc_150k_less.int_rate = inc_150k_less.int_rate.apply(lambda x: float(x.replace("%","")))

## Cleanup Activity on emp_length column
Working on emp_length column to replace null values with some valid value.
I am using earliest_cr_line column to find relevant value for null emp_length.
I am assuming that, when the the first time the credit was opened with LC, the person must be an working somewhere 
as otherwise, LC will not consider giving any loan to him/her.
So, I am extracting year from earliest_cr_line and subtracting it from 2011 that is the year when this data is fetched.
Also I will convert emp_length column from str object to int type. I will remove strings like year, years and
will also replace characters like + and <. Also I am converting <1 year experience to 1 year experience.

In [146]:
inc_150k_less.emp_length.unique()

array(['10+ years', '< 1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '1 year', '6 years', '2 years', '7 years',
       nan], dtype=object)

In [147]:
# creating two columns earliest month and year from earliest credit line
inc_150k_less['earliest_month'] = pd.to_datetime(inc_150k_less["earliest_cr_line"],format="%b-%y").dt.month
inc_150k_less['earliest_year'] = pd.to_datetime(inc_150k_less["earliest_cr_line"],format="%b-%y").dt.year

In [148]:
inc_150k_less[inc_150k_less.earliest_year > 2017][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

,emp_length,annual_inc,earliest_cr_line,earliest_month,earliest_year
1576,NaN,49200.0,Sep-62,9,2062
1764,4 years,63000.0,Sep-68,9,2068
3274,10+ years,72000.0,Sep-67,9,2067
3403,< 1 year,58000.0,Jun-67,6,2067
3595,10+ years,102158.0,Aug-67,8,2067
...,...,...,...,...,...
37288,1 year,40000.0,Dec-66,12,2066
37328,10+ years,93000.0,Oct-68,10,2068
38068,< 1 year,90000.0,Dec-68,12,2068
39549,< 1 year,95000.0,May-63,5,2063


In [149]:
inc_150k_less.earliest_year = inc_150k_less.earliest_year.apply(lambda x: (x-100) if x > 2016 else x)

In [150]:
inc_150k_less[inc_150k_less.earliest_year > 2017][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

,emp_length,annual_inc,earliest_cr_line,earliest_month,earliest_year


In [151]:
inc_150k_less.earliest_year.unique()

array([1985, 1999, 2001, 1996, 2004, 2005, 2007, 1998, 1989, 2003, 1991,
       1993, 1997, 1983, 2002, 1984, 2006, 1987, 1981, 2000, 1994, 1995,
       1992, 1986, 1990, 1988, 1980, 2008, 1979, 1978, 1971, 1972, 1970,
       1969, 1975, 1982, 1962, 1973, 1968, 1974, 1977, 1976, 1967, 1963,
       1965, 1954, 1966, 1964, 1959, 1956, 1946, 1950, 1961], dtype=int64)

In [152]:
# checking updated unique values of emp_length column
inc_150k_less.emp_length.unique()

array(['10+ years', '< 1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '1 year', '6 years', '2 years', '7 years',
       nan], dtype=object)

### Randomly checking values of emp_length in some of the indices before they turn into Not NULL values

In [153]:
inc_150k_less.loc[32608][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

emp_length              NaN
annual_inc          17496.8
earliest_cr_line     Aug-06
earliest_month            8
earliest_year          2006
Name: 32608, dtype: object

In [154]:
inc_150k_less.loc[394][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

emp_length              NaN
annual_inc          24000.0
earliest_cr_line     Feb-05
earliest_month            2
earliest_year          2005
Name: 394, dtype: object

In [155]:
inc_150k_less.loc[422][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

emp_length              NaN
annual_inc          18408.0
earliest_cr_line     Apr-93
earliest_month            4
earliest_year          1993
Name: 422, dtype: object

In [156]:
# function to replace null values of emp_length with the help of earliest_year column 
# that was extracted from earliest_cr_line column
def clean_emp_len(val):
    emp_ear_yr = val.earliest_year
    val.emp_length = int(2011-emp_ear_yr)

    if val.emp_length >=10:
        return 10
    else:
        return int(val.emp_length)

# function to modify not null values in emp_length in order to convert them into int
def mod_emp_len(val):
    #print(2)
    val.emp_length = str(val.emp_length).replace("+","").replace(" years","").replace("< ","").replace(" year","")
    return int(val.emp_length)

In [157]:
###################### calling above functions based upon NULL or NOTNULL values of emp_length column ######################
inc_150k_less['emp_length'] = inc_150k_less.apply(lambda x: mod_emp_len(x) if (pd.notnull(x['emp_length'])) else clean_emp_len(x) ,axis=1)

### Verifying values of emp_length in above checked indices after they turn into Not NULL values

In [158]:
inc_150k_less.loc[32608][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

emp_length                5
annual_inc          17496.8
earliest_cr_line     Aug-06
earliest_month            8
earliest_year          2006
Name: 32608, dtype: object

In [159]:
inc_150k_less.loc[394][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

emp_length                6
annual_inc          24000.0
earliest_cr_line     Feb-05
earliest_month            2
earliest_year          2005
Name: 394, dtype: object

In [160]:
inc_150k_less.loc[422][['emp_length','annual_inc','earliest_cr_line','earliest_month','earliest_year']]

emp_length               10
annual_inc          18408.0
earliest_cr_line     Apr-93
earliest_month            4
earliest_year          1993
Name: 422, dtype: object

In [161]:
# verifying that emp_length column has no NULL values
inc_150k_less.emp_length.isnull().sum()

0

In [163]:
# checking updated unique values of emp_length column
inc_150k_less.emp_length.unique()

array([10,  1,  3,  8,  9,  4,  5,  6,  2,  7], dtype=int64)

### Checking if all the entries are correct when it comes to state zip code and corresponding state

To check if the State is correctly entered or not, I am checking if first 3-digit number of a zip code is present in the zip code range of that state or not. If not, I am moving it to incorrect state data frame and remaining can be used for data analysis

In [164]:
######################
inc_150k_less['state_zip_code'] = inc_150k_less.zip_code.str[0:3].astype(int)

In [165]:
######################
state_zip_codes = { 'AL': [350, 369],
                    'AK': [995, 999],
                    'AZ': [850, 865],
                    'AR': [716, 729],
                    'CA': [900, 961],
                    'CO': [800, 816],
                    'CT': [60, 69],
                    'DC': [200,200],
                    'DE': [197, 199],
                    'FL': [320, 349],
                    'GA': [300, 399],
                    'HI': [967, 968],
                    'ID': [832, 838],
                    'IL': [600, 629],
                    'IN': [460, 479],
                    'IA': [500, 528],
                    'KS': [660, 679],
                    'KY': [400, 427],
                    'LA': [700, 714],
                    'ME': [39, 49],
                    'MD': [205, 219],
                    'MA': [10, 55],
                    'MI': [480, 499],
                    'MN': [550, 567],
                    'MS': [386, 397],
                    'MO': [630, 658],
                    'MT': [590, 599],
                    'NE': [680, 693],
                    'NV': [889, 898],
                    'NH': [30, 38],
                    'NJ': [70, 89],
                    'NM': [870, 884],
                    'NY': [5, 149],
                    'NC': [270, 289],
                    'ND': [580, 588],
                    'OH': [430, 459],
                    'OK': [730, 749],
                    'OR': [970, 979],
                    'PA': [150, 196],
                    'RI': [28, 29],
                    'SC': [290, 299],
                    'SD': [570, 577],
                    'TN': [370, 385],
                    'TX': [733, 885],
                    'UT': [840, 847],
                    'VT': [50, 59],
                    'VA': [201, 246],
                    'WA': [980, 994],
                    'WV': [247, 268],
                    'WI': [530, 549],
                    'WY': [820, 834]
                  }

In [166]:
######################
incorrect_state_codes = pd.DataFrame()
loan_valid_state_codes = pd.DataFrame()

In [167]:
######################
for i in state_zip_codes.keys():
    #print(str(i) + " " +
    #str(len(loan_without_null_coulumns[(loan_without_null_coulumns.addr_state == i) & ~(loan_without_null_coulumns.state_code.between(state_zip_codes[i][0],state_zip_codes[i][1]))])) +
    #" " + str(len(loan_without_null_coulumns[(loan_without_null_coulumns.addr_state == i) & (loan_without_null_coulumns.state_code.between(state_zip_codes[i][0],state_zip_codes[i][1]))]))
    #+ " " + str(len(loan_without_null_coulumns[loan_without_null_coulumns.addr_state == i])))
    #print(str(i) + " " + str(len(loan_without_null_coulumns[loan_without_null_coulumns.addr_state == i])))
    incorrect_state_codes = incorrect_state_codes.append(inc_150k_less[(inc_150k_less.addr_state == i) & ~(inc_150k_less.state_zip_code.between(state_zip_codes[i][0],state_zip_codes[i][1]))])
    loan_valid_state_codes = loan_valid_state_codes.append(inc_150k_less[(inc_150k_less.addr_state == i) & (inc_150k_less.state_zip_code.between(state_zip_codes[i][0],state_zip_codes[i][1]))])


In [168]:
incorrect_state_codes.shape

(82, 21)

In [169]:
loan_valid_state_codes.shape

(36913, 21)

In [170]:
incorrect_state_codes[['addr_state', 'zip_code', 'state_zip_code']]

,addr_state,zip_code,state_zip_code
22597,AZ,727xx,727
24725,AZ,925xx,925
7244,AR,864xx,864
17467,AR,857xx,857
36499,AR,711xx,711
...,...,...,...
21236,WA,941xx,941
37212,WA,838xx,838
9491,WV,285xx,285
26204,WV,222xx,222


In [171]:
state_zip_codes

{'AL': [350, 369],
 'AK': [995, 999],
 'AZ': [850, 865],
 'AR': [716, 729],
 'CA': [900, 961],
 'CO': [800, 816],
 'CT': [60, 69],
 'DC': [200, 200],
 'DE': [197, 199],
 'FL': [320, 349],
 'GA': [300, 399],
 'HI': [967, 968],
 'ID': [832, 838],
 'IL': [600, 629],
 'IN': [460, 479],
 'IA': [500, 528],
 'KS': [660, 679],
 'KY': [400, 427],
 'LA': [700, 714],
 'ME': [39, 49],
 'MD': [205, 219],
 'MA': [10, 55],
 'MI': [480, 499],
 'MN': [550, 567],
 'MS': [386, 397],
 'MO': [630, 658],
 'MT': [590, 599],
 'NE': [680, 693],
 'NV': [889, 898],
 'NH': [30, 38],
 'NJ': [70, 89],
 'NM': [870, 884],
 'NY': [5, 149],
 'NC': [270, 289],
 'ND': [580, 588],
 'OH': [430, 459],
 'OK': [730, 749],
 'OR': [970, 979],
 'PA': [150, 196],
 'RI': [28, 29],
 'SC': [290, 299],
 'SD': [570, 577],
 'TN': [370, 385],
 'TX': [733, 885],
 'UT': [840, 847],
 'VT': [50, 59],
 'VA': [201, 246],
 'WA': [980, 994],
 'WV': [247, 268],
 'WI': [530, 549],
 'WY': [820, 834]}